In [1]:
import requests

In [2]:
from bedrock.client.client import BedrockAPI

In [3]:
SERVER = "http://localhost:81/"

In [4]:
api = BedrockAPI(SERVER)

In [5]:
filepath =  '/home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv'

In [6]:
ingest_id = 'opals.spreadsheet.Spreadsheet.Spreadsheet'
resp = api.put_source('Rand2001',ingest_id, 'default',{'file':open(filepath, "rb")})
if resp.status_code == 201:
    print 'Source {0} successfully uploaded'.format(filepath)
else:
    print 'Failed to update {0}'.format(filepath)

INFO: 2017-09-05 18:20:11,708: putting source to: http://localhost:81/dataloader/sources/Rand2001/opals.spreadsheet.Spreadsheet.Spreadsheet/default/


Failed to update /home/atam6/git/bedrock-core/examples/RAND2011study/Rand2011PNAS_cooperation_data.csv


In [7]:
available_sources = api.list("dataloader","sources").json()
source = (s for s in available_sources if s["name"] == "Rand2001").next()
source_id = source["src_id"]
endpoint = api.endpoint("dataloader","sources/%s/explore/" % source_id)
resp = requests.get(endpoint)

INFO: 2017-09-05 18:20:11,884: Listing: http://localhost:81/dataloader/sources/


In [8]:
resp = api.create_matrix(source_id, 'rand_mtx')
mtx = resp[0]
matrix_id = mtx['id']
print(mtx)
#resp

{u'src_id': u'2876d3ecda384e6ca195fefc0de078e9', u'name': u'rand_mtx', u'created': u'2017-09-05 22:20:12.464150', u'outputs': [u'features_original.txt', u'features.txt', u'matrix.csv'], u'id': u'994c343acccc46ce81225e7fe2fce07d', u'rootdir': u'/opt/bedrock/dataloader/data/2876d3ecda384e6ca195fefc0de078e9/994c343acccc46ce81225e7fe2fce07d/', u'filters': {u'fluid_dummy': {}, u'previous_decision': {}, u'group_size': {}, u'decision0d1c': {}, u'playerid': {}, u'_Icondition_4': {}, u'_Icondition_3': {}, u'_Icondition_2': {}, u'sessionnum': {}, u'num_neighbors': {}, u'round_num': {}, u'condition': {}}, u'mat_type': u'csv'}


In [9]:
resp = api.analytic('opals.stan.Stan.Stan_GLM')
if resp.json():
    print("Stan_GLM Opal Installed!")
else:
    print("Stan_GLM Opal Not Installed!")

Stan_GLM Opal Installed!


In [10]:
resp

<Response [200]>

## Run Stan_GLM

In [11]:
analytic_id = "opals.stan.Stan.Stan_GLM"
inputData = {
    'matrix.csv': mtx,
    'features.txt': mtx
}

paramsData = [
            { "name" : "Formula", "attrname" : "formula", "value" : "decision0d1c ~ condition", "type" : "input" },
            { "name" : "GLM family", "attrname" : "family", "value" : 'binomial(link = "logit")', "type" : "input" },
            { "name" : "chains", "attrname" : "chains" , "value" : "3", "type" : "input"},
            { "name" : "iter", "attrname" : "iter" , "value" : "3000", "type" : "input"},
            { "name" : "prior", "attrname" : "prior" , "value" : "", "type" : "input"},
            { "name" : "prior_intercept", "attrname" : "prior_intercept" , "value" : "", "type" : "input"}
        ]

result_mtx = api.run_analytic(analytic_id, mtx, 'rand_stan_test', input_data=inputData, parameter_data=paramsData)

In [12]:
print result_mtx

{u'src_id': u'994c343acccc46ce81225e7fe2fce07d', u'name': u'rand_stan_test', u'parameters': [{u'attrname': u'formula', u'type': u'input', u'name': u'Formula', u'value': u'decision0d1c ~ condition'}, {u'attrname': u'family', u'type': u'input', u'name': u'GLM family', u'value': u'binomial(link = "logit")'}, {u'attrname': u'chains', u'type': u'input', u'name': u'chains', u'value': u'3'}, {u'attrname': u'iter', u'type': u'input', u'name': u'iter', u'value': u'3000'}, {u'attrname': u'prior', u'type': u'input', u'name': u'prior', u'value': u''}, {u'attrname': u'prior_intercept', u'type': u'input', u'name': u'prior_intercept', u'value': u''}], u'created': u'2017-09-05 22:21:18.540440', u'outputs': [u'prior_summary.txt', u'summary.txt'], u'analytic_id': u'opals.stan.Stan.Stan_GLM', u'rootdir': u'/opt/bedrock/analytics/data/994c343acccc46ce81225e7fe2fce07d/2f632270bb6b4b5fba0750542e562b72/', u'id': u'2f632270bb6b4b5fba0750542e562b72'}


## Get Stan_GLM outputs

In [13]:
result_mtx['src_id']

u'994c343acccc46ce81225e7fe2fce07d'

In [14]:
result_mtx['id']

u'2f632270bb6b4b5fba0750542e562b72'

In [15]:
import requests

In [16]:
base_url = "http://localhost:81/analytics/results/{0}/{1}/download/prior_summary.txt/prior_summary.txt/"

In [17]:
url = base_url.format(result_mtx['src_id'],result_mtx['id'])

In [18]:
response = requests.get(url)

In [24]:
print response.text


Model Info:

 function:  stan_glm
 family:    binomial [logit]
 formula:   "decision0d1c~round_num"
 algorithm: sampling
 priors:    see help('prior_summary')
 sample:    4500 (posterior sample size)
 num obs:   3876

Estimates:
                mean    sd      2.5%    25%     50%     75%     97.5%
(Intercept)       0.4     0.1     0.3     0.4     0.4     0.5     0.6
round_num10      -1.6     0.3    -2.2    -1.8    -1.6    -1.4    -1.0
round_num11      -1.4     0.3    -2.0    -1.6    -1.4    -1.2    -0.9
round_num2        0.1     0.1    -0.1     0.0     0.1     0.1     0.3
round_num3       -0.2     0.1    -0.4    -0.3    -0.2    -0.1     0.0
round_num4       -0.3     0.1    -0.5    -0.4    -0.3    -0.2    -0.1
round_num5       -0.5     0.1    -0.8    -0.6    -0.5    -0.4    -0.3
round_num6       -0.7     0.1    -1.0    -0.8    -0.7    -0.6    -0.4
round_num7       -0.7     0.2    -1.0    -0.8    -0.7    -0.6    -0.4
round_num8       -0.8     0.2    -1.1    -0.9    -0.8    -0.7    -0.5


In [20]:
base_url = "http://localhost:81/analytics/results/{0}/{1}/download/summary.txt/summary.txt/"

In [21]:
url = base_url.format(result_mtx['src_id'],result_mtx['id'])

In [22]:
response = requests.get(url)

In [25]:
print response.text


Model Info:

 function:  stan_glm
 family:    binomial [logit]
 formula:   "decision0d1c~round_num"
 algorithm: sampling
 priors:    see help('prior_summary')
 sample:    4500 (posterior sample size)
 num obs:   3876

Estimates:
                mean    sd      2.5%    25%     50%     75%     97.5%
(Intercept)       0.4     0.1     0.3     0.4     0.4     0.5     0.6
round_num10      -1.6     0.3    -2.2    -1.8    -1.6    -1.4    -1.0
round_num11      -1.4     0.3    -2.0    -1.6    -1.4    -1.2    -0.9
round_num2        0.1     0.1    -0.1     0.0     0.1     0.1     0.3
round_num3       -0.2     0.1    -0.4    -0.3    -0.2    -0.1     0.0
round_num4       -0.3     0.1    -0.5    -0.4    -0.3    -0.2    -0.1
round_num5       -0.5     0.1    -0.8    -0.6    -0.5    -0.4    -0.3
round_num6       -0.7     0.1    -1.0    -0.8    -0.7    -0.6    -0.4
round_num7       -0.7     0.2    -1.0    -0.8    -0.7    -0.6    -0.4
round_num8       -0.8     0.2    -1.1    -0.9    -0.8    -0.7    -0.5
